In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
path_trn = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

path_tst = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

df_trn = pd.read_csv(path_trn, names=CSV_COLUMN_NAMES, header=0)
df_tst = pd.read_csv(path_tst, names=CSV_COLUMN_NAMES, header=0)

In [5]:
df_trn.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
y_trn = df_trn.pop('Species')
y_tst = df_tst.pop('Species')

In [7]:
df_trn.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [8]:
def input_fn(features, labels, training=True, batch_size=256):
    """Input function for Tensorflow Estimators"""
    # Convert inputs to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    if training:
        dataset = dataset.shuffle(1024).repeat()

    return dataset.batch(batch_size)

In [9]:
feature_columns = list()
for key in df_trn.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [10]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 10],
    n_classes=3)

W0610 15:37:15.751109  3296 estimator.py:1811] Using temporary folder as model directory: C:\Users\Michael\AppData\Local\Temp\tmpwncoaift


In [11]:
classifier.train(input_fn= lambda: input_fn(df_trn, y_trn, training=True), steps=5000)

W0610 15:37:17.322477  3296 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0610 15:37:18.016971  3296 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\head\base_head.py:574: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0610 15:37:18.242588  3296 deprecation.py:506] From C:\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\adagrad.py:105: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call

In [13]:
eval_result = classifier.evaluate(input_fn = lambda : input_fn(df_tst, y_tst, training=False))

W0610 15:38:58.295886  3296 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [18]:
eval_result

{'accuracy': 0.53333336,
 'average_loss': 0.9079215,
 'loss': 0.9079215,
 'global_step': 5000}

In [19]:
acc_tst = eval_result['accuracy']
print(f'Test set accuracy={acc_tst:0.3f}')

Test set accuracy=0.533
